In [28]:
import pandas as pd, numpy as np

In [29]:
# merge hadicappers' history bets into one dataframe
names = [
    'Smart',
    'Bartley',
    'Aronson',
    'Burns',
    'Barone', # not in Ji's original list
    'Bitler',
    'Power',
    'Ross',
    'Diamond',
    'Trapp',
    'Sports',
    'Eddie',
    'Schule',
    'DAmico',
    'Duffy',
    'Thomas',
    'Hunter',
    'Compeau', # not in Ji's original list
    'Syndicate',
    'Lundin',
    'Simulator',
    'Wilson',
    'Monohan',
    'Vinceletti',
    'Rickenbach',
    'Higgs',
    'Nover',
    'Brown',
    'Karpinski',
    'Rogers'
    ]

df = pd.DataFrame()
for i in range(1,31): 
    newdf = pd.read_pickle('HC' + str(i) + '.gz', compression='gzip')
    
    # if you want the last name of the HC, switch to use names array
    #newdf['Handicapper'] = names[i-1]
    newdf['Handicapper'] = 'HC'+str(i)
    df = df.append(newdf, ignore_index=True)

In [5]:
df.head()

,League,GameKey,GameTime,Home,Away,Bet,On,Juice,Type,Result,MLBet,MLWinner,Handicapper
0,MLB,3a4f8adc60ca77db09c8a65061322a2e,2018-04-11 13:10:00,MINNESOTA,HOUSTON,TOTAL,OVER,-101,Premium,Win,,,HC1
1,NBA,f87ad4c949d7f3c1e61011d57b2bd224,2018-04-10 21:00:00,UTAH,GOLDEN STATE,POINT SPREAD,UTAH,-105,Free,Win,,,HC1
2,MLB,fc82ea958a93190a5c81532ba3959084,2018-04-10 20:40:00,COLORADO,SAN DIEGO,MONEY LINE,COLORADO,-152,Premium,Loss,Home,Away,HC1
3,MLB,d679ec8ea96d147f878516455716fa10,2018-04-10 19:05:00,WASHINGTON,ATLANTA,TOTAL,OVER,+110,Premium,Loss,,,HC1
4,MLB,d655303abf834b0dfbe66dc6c1554175,2018-04-10 19:05:00,BALTIMORE,TORONTO,POINT SPREAD,BALTIMORE,-145,Premium,Win,,,HC1


In [80]:

# add column for # of wins in the past 30 games
# add column for % of wins in the past 30 games
# if game is 0, assume win % is 50%
# add column for winning steak
def win_past_30_all_sport(df):
    df=df.sort_values(['Handicapper','GameTime'], ascending=[1,1])
    df=df.reset_index(drop=True)
    win_past_30=[]
    win_past_30_all_sport_percent=[]
    win_tracker=[]
    win_steak=[]
    win=0
    lose=0
    win_steak=[]
    
    pointer=0
    last=df.League.count()
    HC=df['Handicapper'][0]
    while pointer < last:
        if df['Handicapper'][pointer]!=HC:
            HC=df['Handicapper'][pointer]
            win_past_30.append(0)
            win_past_30_all_sport_percent.append(0.5)
            win_steak.append(0)
        elif pointer == 0:
            win_past_30.append(0)
            win_past_30_all_sport_percent.append(0.5)
            win_steak.append(0)
        else: 
            win_past_30.append(sum(win_tracker))
            mean = sum(win_tracker)/len(win_tracker)
            win_past_30_all_sport_percent.append(mean)
            if (win>lose):
                win_steak.append(win)
            else:
                win_steak.append(-lose)

        
        if df['Result'][pointer]=='Win':
            win_tracker.insert(0,1)
            win+=1
            lose=0
        elif df['Result'][pointer]=='Push':
            win_tracker.insert(0,1)
        else:
            win_tracker.insert(0,0)
            win=0
            lose+=0
            
            
        
        if len(win_tracker)>30:
            win_tracker.pop()
        pointer+=1

    df['win_past_30_all_sport']=win_past_30
    df['win_past_30_all_sport_percent']=win_past_30_all_sport_percent
    df['win_steak_all_sport']=win_steak
    return df
    
# add column for # of wins in the past 30 games for specific sport(league)
# add column for % of wins in the past 30 games for specific sport(league)
# if game is 0, assume win % is 50%
# add column for winning steak for specific sport(league) 
def win_past_30_specific_sport(df):
    df=df.sort_values(['Handicapper','League','GameTime'], ascending=[1,1,1])
    df=df.reset_index(drop=True)
    win_past_30=[]
    win_past_30_specific_sport_percent=[]
    win_tracker=[]
    pointer=0
    win_steak=[]
    win=0
    lose=0
    last=df.League.count()
    HC=df['Handicapper'][0]
    League=df['League'][0]
    while pointer < last:
        if (df['Handicapper'][pointer]!=HC) or (df['League'][pointer]!=League):
            HC=df['Handicapper'][pointer]
            League=df['League'][pointer]
            win_tracker=[]
            win_past_30.append(0)
            win_past_30_specific_sport_percent.append(0.5)
            win_steak.append(0)
        elif pointer == 0:
            win_past_30.append(0)
            win_past_30_specific_sport_percent.append(0.5)
            win_steak.append(0)
        else: 
            win_past_30.append(sum(win_tracker))
            mean = sum(win_tracker)/len(win_tracker)
            win_past_30_specific_sport_percent.append(mean)
            if (win>lose):
                win_steak.append(win)
            else:
                win_steak.append(-lose)
            
            
        if df['Result'][pointer]=='Win':
            win_tracker.insert(0,1)
            win+=1
            lose=0
        elif df['Result'][pointer]=='Push':
            win_tracker.insert(0,1)
        else:
            win_tracker.insert(0,0)
            win=0
            lose+=0
            
            
        if len(win_tracker)>30:
            win_tracker.pop()

        pointer+=1
    df['win_past_30_specific_sport']=win_past_30
    df['win_past_30_specific_sport_percent']=win_past_30_specific_sport_percent
    df['win_steak_specific_sport']=win_steak
    return df


## convert juice to risk level
## if juice more than 120, then we are betting on underdog
## if juice less than -140, then we are betting on favorite
## I consider everything else is 'normal'
def bet_risk(df):
    pointer=0
    bet_risk=[]
    last=df.League.count()
    while pointer < last:
        if int(df['Juice'][pointer])>120:
            bet_risk.append('underdog')
        elif int(df['Juice'][pointer])<-140:
            bet_risk.append('favorite')
        else:
            bet_risk.append('normal')
        pointer+=1
    df['bet_risk']=bet_risk
    return df

# add column for # of wins in the past 30 games for specific risk
# add column for % of wins in the past 30 games for specific risk
# if game is 0, assume win % is 50%
# add column for winning steak for specific risk
def win_past_30_specific_risk(df):
    df=df.sort_values(['Handicapper','bet_risk','GameTime'], ascending=[1,1,1])
    df=df.reset_index(drop=True)
    win_past_30=[]
    win_past_30_specific_risk_percent=[]
    win_tracker=[]
    pointer=0
    last=df.League.count()
    HC=df['Handicapper'][0]
    Risk=df['bet_risk'][0]
    while pointer < last:
        if (df['Handicapper'][pointer]!=HC) or (df['bet_risk'][pointer]!=Risk):
            HC=df['Handicapper'][pointer]
            Risk=df['bet_risk'][pointer]
            win_tracker=[]
            win_past_30.append(0)
            win_past_30_specific_risk_percent.append(0.5)
        elif pointer == 0:
            win_past_30.append(0)
            win_past_30_specific_risk_percent.append(0.5)
        else: 
            win_past_30.append(sum(win_tracker))
            mean = sum(win_tracker)/len(win_tracker)
            win_past_30_specific_risk_percent.append(mean)
        if df['Result'][pointer]=='Win':
            win_tracker.insert(0,1)
        else:
            win_tracker.insert(0,0)
        if len(win_tracker)>30:
            win_tracker.pop()

        pointer+=1
    df['win_past_30_specific_risk']=win_past_30
    df['win_past_30_specific_risk_percent']=win_past_30_specific_risk_percent
    return df



In [81]:
df = win_past_30_all_sport(df)
df = win_past_30_specific_sport(df)
df = bet_risk(df)
df = win_past_30_specific_risk(df)
df.to_csv('debug.csv', encoding='utf-8-sig')

